# Building a text classification model, with the inclusion of data extractions from PDF files

### 1. Import Python libraries

In [1]:
# Reading in relevant libraries from python
import re
import numpy as np 
import pandas as pd
import string
from PDF_EXTRACT import convert
# pip install pdfminer.six
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('stopwords')

# define set of common words known as stopwords
letters = string.ascii_letters
stopwords_set = set(stopwords.words('english')) 
# define list of other characters and symbols
extra_characters = ["!@#$%^&)(_-+=|\][}{;:'?/><.,~`"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WZCHE\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


### 2. Import dataset, assigning x-features and y-target, followed by data pre-processing

In [3]:
# define page numbers in PDF to be read in
pages_1 = [8,9] # starting page number and final page number
pages_2 = [10,11]
# data extraction from PDF file
extracted_text_plant = convert('datasets/Text data.pdf',pages_1)
extracted_text_notplant = convert('datasets/Text data.pdf',pages_2)
print(extracted_text_plant)
print('-------------------')
print(extracted_text_notplant)

 

 

Annex B 

 
 

EXAMPLES OF ITEMS THAT CAN BE CONSIDERED AS PLANT 

1  Air conditioning and other mechanical installation 

 

2  Artwork  and  tapestries,  posters,  paintings,  picture  frames,  demountable  décor  pillar, 

demountable  décor column and  ornamental  features,  including  decorative  finishes  and 

murals.  These stand-alone objects are generally found to serve a business function in 

hospitality-related businesses, example clubs, restaurants and hotels. For items that are 

quasi-structural, i.e. demountable with separate identity, e.g. mirrors, décor, a functional 

analysis is adopted if a legitimate function is served, and the item would be treated as 

plant even if there are doubts as to whether the premises test is satisfied. In other non-

hospitality  related  businesses,  taxpayer  need  to  demonstrate  that  these  are  plant  and 

machinery of the trade. 

3  Books that are tools of the trade and used in the day-to-day operation of the profession

#### For plant-related dataset 

In [4]:
# locating index number of which each item is located 
index_list, length_content_list,training_data_plant = list(), list(), list()
for i in range(len(extracted_text_plant)):
    if (extracted_text_plant[i]).isnumeric() is True:
        index_list.append(i)

# accounting for the additional spaces to the indexes
index_list = [item+2 for item in index_list] 

for i in range(len(index_list)-1):
    length_content_list.append(index_list[i+1]-index_list[i])
length_content_list.append(len(extracted_text_plant)-index_list[len(index_list)-1])

# Data cleaning step for each item by removing the following: 
# (a) remove empty spaces in the front and back; 
# (b) remove digits; 
# (c) remove other characters and symbols
for count in range(len(index_list)):
    total_count = length_content_list[count]
#     sub_list = list()
    text = ''
    i = 0
    while True:
        text += extracted_text_plant[index_list[count]+i]
        i += 1
        if i >= total_count:
            text = text.rstrip()
            text = text.replace('\n','') # replace empty space with no characters
            text = re.sub(r'\d+', '', text) # substitute digits with no characters
            text = text.rstrip('\n')
            text = text.strip()
            for item in extra_characters:
                text = text.replace(item,'')
#             sub_list.append(text)
            training_data_plant.append(text)
            break
print(training_data_plant)

['Air conditioning and other mechanical installation', 'Artwork  and  tapestries,  posters,  paintings,  picture  frames,  demountable  décor  pillar, demountable  décor column and  ornamental  features,  including  decorative  finishes  and murals.  These stand-alone objects are generally found to serve a business function in hospitality-related businesses, example clubs, restaurants and hotels. For items that are quasi-structural, i.e. demountable with separate identity, e.g. mirrors, décor, a functional analysis is adopted if a legitimate function is served, and the item would be treated as plant even if there are doubts as to whether the premises test is satisfied. In other non-hospitality  related  businesses,  taxpayer  need  to  demonstrate  that  these  are  plant  and machinery of the trade.', 'Books that are tools of the trade and used in the day-to-day operation of the profession', 'Blinds and curtains', 'Building automation and control systems', 'Built-in-storage units and 

In [5]:
# remove stopwords and all lists of no characters
training_data_plant_cleaned = list()
for item1 in training_data_plant:
    word_list = list()
    for word in item1.split(' '):
        word_list.append(word)
    training_data_plant_cleaned.append(word_list)
print(training_data_plant_cleaned)
print('')
print('-------------------------')
print('')

training_data_plant_cleaned_2 = list()
for item2 in training_data_plant_cleaned:
    word_list = list()
    for item3 in item2:
        if item3 not in stopwords_set and item3 != '':
            word_list.append(item3)
    training_data_plant_cleaned_2.append(word_list)
training_data_plant_cleaned_2 = [e for e in training_data_plant_cleaned_2 if e and e != list('.')]
print(training_data_plant_cleaned_2)

[['Air', 'conditioning', 'and', 'other', 'mechanical', 'installation'], ['Artwork', '', 'and', '', 'tapestries,', '', 'posters,', '', 'paintings,', '', 'picture', '', 'frames,', '', 'demountable', '', 'décor', '', 'pillar,', 'demountable', '', 'décor', 'column', 'and', '', 'ornamental', '', 'features,', '', 'including', '', 'decorative', '', 'finishes', '', 'and', 'murals.', '', 'These', 'stand-alone', 'objects', 'are', 'generally', 'found', 'to', 'serve', 'a', 'business', 'function', 'in', 'hospitality-related', 'businesses,', 'example', 'clubs,', 'restaurants', 'and', 'hotels.', 'For', 'items', 'that', 'are', 'quasi-structural,', 'i.e.', 'demountable', 'with', 'separate', 'identity,', 'e.g.', 'mirrors,', 'décor,', 'a', 'functional', 'analysis', 'is', 'adopted', 'if', 'a', 'legitimate', 'function', 'is', 'served,', 'and', 'the', 'item', 'would', 'be', 'treated', 'as', 'plant', 'even', 'if', 'there', 'are', 'doubts', 'as', 'to', 'whether', 'the', 'premises', 'test', 'is', 'satisfied.',

In [6]:
# create list of y-targets which quantity corresponds to the number of items 
y_target_plant = list()
for i in range(len(training_data_plant_cleaned_2)):
    y_target_plant.append('Plant Classification')

print(training_data_plant_cleaned_2)
print('')
print(y_target_plant)

[['Air', 'conditioning', 'mechanical', 'installation'], ['Artwork', 'tapestries,', 'posters,', 'paintings,', 'picture', 'frames,', 'demountable', 'décor', 'pillar,', 'demountable', 'décor', 'column', 'ornamental', 'features,', 'including', 'decorative', 'finishes', 'murals.', 'These', 'stand-alone', 'objects', 'generally', 'found', 'serve', 'business', 'function', 'hospitality-related', 'businesses,', 'example', 'clubs,', 'restaurants', 'hotels.', 'For', 'items', 'quasi-structural,', 'i.e.', 'demountable', 'separate', 'identity,', 'e.g.', 'mirrors,', 'décor,', 'functional', 'analysis', 'adopted', 'legitimate', 'function', 'served,', 'item', 'would', 'treated', 'plant', 'even', 'doubts', 'whether', 'premises', 'test', 'satisfied.', 'In', 'non-hospitality', 'related', 'businesses,', 'taxpayer', 'need', 'demonstrate', 'plant', 'machinery', 'trade.'], ['Books', 'tools', 'trade', 'used', 'day-to-day', 'operation', 'profession'], ['Blinds', 'curtains'], ['Building', 'automation', 'control', 

In [6]:
# create a compact list of lists whereby each sub-list contain the full sentence for each item
training_data_plant_cleaned_3 = list()
for item1 in training_data_plant_cleaned_2:
    dummy_list = list()
    dummy_string = ''
    for item2 in item1:
        dummy_string = dummy_string + item2
        dummy_string = dummy_string + ' '
    dummy_list.append(dummy_string.strip())
    training_data_plant_cleaned_3.append(dummy_list)

print(training_data_plant_cleaned_3)

[['Air conditioning mechanical installation'], ['Artwork tapestries, posters, paintings, picture frames, demountable décor pillar, demountable décor column ornamental features, including decorative finishes murals. These stand-alone objects generally found serve business function hospitality-related businesses, example clubs, restaurants hotels. For items quasi-structural, i.e. demountable separate identity, e.g. mirrors, décor, functional analysis adopted legitimate function served, item would treated plant even doubts whether premises test satisfied. In non-hospitality related businesses, taxpayer need demonstrate plant machinery trade.'], ['Books tools trade used day-to-day operation profession'], ['Blinds curtains'], ['Building automation control systems'], ['Built-in-storage units dispensers'], ['Carpets carpet tiles'], ['Central grease removal system performs special function business'], ['Cold water storage tanks form part general water supply'], ['Containers used carriage goods

#### For non-plant related data

In [7]:
# data cleaning for non-plant classification
training_data_non_plant = list()
index_list = list()
length_content_list = list()

for i in range(len(extracted_text_notplant)):
    if (extracted_text_notplant[i]).isnumeric() is True:
        index_list.append(i)
index_list = [item+2 for item in index_list]
for i in range(len(index_list)-1):
    length_content_list.append(index_list[i+1]-index_list[i])
length_content_list.append(len(extracted_text_notplant)-index_list[len(index_list)-1])

for count in range(len(index_list)):
    total_count = length_content_list[count]
#     sub_list = list()
    text = ''
    i = 0
    while True:
        text += extracted_text_notplant[index_list[count]+i]
        i += 1
        if i >= total_count:
            text = text.rstrip()
            text = text.replace('\n','')
            text = re.sub(r'\d+', '', text)
            text = text.rstrip('\n')
            text = text.strip()
#             sub_list.append(text)
            training_data_non_plant.append(text)
            break

empty_list = [[''],['.']]
for item in training_data_non_plant:
    if item in empty_list:
        training_data_non_plant.remove(item)
training_data_non_plant.pop()
for item in training_data_non_plant:
    if item == [''] or item == ['.']:
        training_data_non_plant.remove(item)
training_data_non_plant.pop()

# remove stopwords
training_data_non_plant_cleaned = list()
for item1 in training_data_non_plant:
    word_list = list()
    for word in item1.split(' '):
        word_list.append(word)
    training_data_non_plant_cleaned.append(word_list)

training_data_non_plant_cleaned_2 = list()
for item2 in training_data_non_plant_cleaned:
    word_list = list()
    for item3 in item2:
        if item3 not in stopwords_set and item3 != '':
            word_list.append(item3)
    training_data_non_plant_cleaned_2.append(word_list)
training_data_non_plant_cleaned_2 = [e for e in training_data_non_plant_cleaned_2 if e and e != list('.')]

y_target_non_plant = list()
for i in range(len(training_data_non_plant_cleaned_2)):
    y_target_non_plant.append('Non-Plant Classification')
    
print(training_data_non_plant_cleaned_2)
print('')
print(y_target_non_plant)
print('')

training_data_non_plant_cleaned_3 = list()
for item1 in training_data_non_plant_cleaned_2:
    dummy_list = list()
    dummy_string = ''
    for item2 in item1:
        dummy_string = dummy_string + item2
        dummy_string = dummy_string + ' '
    dummy_list.append(dummy_string.strip())
    training_data_non_plant_cleaned_3.append(dummy_list)

print(training_data_non_plant_cleaned_3)
print('')

[['Awning'], ['Doors/door', 'handles/door', 'stopper/mechanical', 'door', 'closer/automatic', 'sliding', 'door'], ['Dormitory', 'used', 'temporary', 'labour', 'quarters', 'erected', 'using', 'pre-fabricated', 'structure', 'demountable', 'even', 'connected', 'bolts', 'nuts.', 'However,', 'dormitory', 'qualify', 'plant', 'erected', 'site', 'short-term', 'lease', 'dormitory', 'operator.', 'The', 'dormitory', 'must', 'constructed', 'temporary', 'ephemeral', 'materials', 'facilitate', 'intention', 'temporarily', 'locate', 'move', 'place', 'place.'], ['Floor', 'tiling/raised', 'floors,', 'false', 'ceiling,', 'ceiling', 'boards', 'ceiling', 'work'], ['Fountain', 'water', 'features', '-', 'Fountains', 'water', 'features', 'treated', 'structural', 'plant.', 'However,', 'mechanical', 'parts', 'may', 'considered', 'machinery.'], ['Garbage', 'waste', 'disposal', 'system,', 'statutory', 'requirement'], ['Gates'], ['Installation', 'gas', 'system', 'general', 'gas', 'supply'], ['Landscaping'], ['Gene

####  Analysing the respective words for both plant and non-plant datasets

In [8]:
# creating dictionaries to count the common words
dict_plant_data = dict()
for item1 in training_data_plant_cleaned_2:
    for item2 in item1:
        if item2 in dict_plant_data.keys():
            dict_plant_data[item2] += 1
        else:
            dict_plant_data[item2] = 1
# print(dict_plant_data)

print('List of common items with their respective frequency for plant category')
result_plant = sorted(dict_plant_data.items(), key = 
             lambda kv:(kv[1], kv[0]))
print((result_plant))
print('')

dict_non_plant_data = dict()
for item1 in training_data_non_plant_cleaned_2:
    for item2 in item1:
        if item2 in dict_non_plant_data.keys():
            dict_non_plant_data[item2] += 1
        else:
            dict_non_plant_data[item2] = 1
# print(dict_non_plant_data)

print('List of common items with their respective frequency for non-plant category')
result_non_plant = sorted(dict_non_plant_data.items(), key = 
             lambda kv:(kv[1], kv[0]))
print(result_non_plant) 

print('')
print('List of common items with their respective frequency for plant and non-plant categories')
dict_plant_and_nonplant_data = dict_plant_data.keys() & dict_non_plant_data.keys()
print(dict_plant_and_nonplant_data) # Finding keys in common

List of common items with their respective frequency for plant category
[('(but', 1), ('(goods/commercial', 1), ('(lift', 1), ('(ornamental', 1), ('Air', 1), ('Artwork', 1), ('Blinds', 1), ('Books', 1), ('Building', 1), ('Built-in-storage', 1), ('Carpets', 1), ('Central', 1), ('Cold', 1), ('Containers', 1), ('Data', 1), ('Escalator', 1), ('Exit/emergency', 1), ('Fire', 1), ('For', 1), ('Furniture', 1), ('Generator', 1), ('Hot', 1), ('ITA)', 1), ('In', 1), ('Industrial', 1), ('Lifts,', 1), ('Lightning', 1), ('Mechanical', 1), ('Mirrors', 1), ('Motor', 1), ('Movable/demountable', 1), ('Office', 1), ('Partitions', 1), ('Playground', 1), ('Public', 1), ('Pumps', 1), ('Ramps', 1), ('Security', 1), ('Showcase', 1), ('Signboard', 1), ('Stand', 1), ('Switchboard,', 1), ('These', 1), ('Vehicle', 1), ('Window', 1), ('accessories', 1), ('address', 1), ('adopted', 1), ('air-conditioning', 1), ('alarm', 1), ('alone', 1), ('analysis', 1), ('another', 1), ('appliances)', 1), ('automation', 1), ('bicy

In [9]:
plant_data, non_plant_data = list(), list()

for item in training_data_plant_cleaned_2:
    string = ''
    for sub_item in item:
        if sub_item not in dict_plant_and_nonplant_data:
            string = sub_item + ' ' + string
    string.rstrip()
    plant_data.append(string)

for item in training_data_non_plant_cleaned_2:
    string = ''
    for sub_item in item:
        if sub_item not in dict_plant_and_nonplant_data:
            string = sub_item + ' ' + string
    string.rstrip()
    non_plant_data.append(string)
    
print(plant_data)
print('')
print(non_plant_data)

['installation conditioning Air ', 'machinery demonstrate need taxpayer businesses, related non-hospitality In satisfied. test premises doubts item served, function legitimate adopted analysis functional décor, mirrors, e.g. identity, separate i.e. quasi-structural, hotels. restaurants clubs, example businesses, hospitality-related function business serve found generally objects stand-alone These murals. decorative including features, ornamental column décor pillar, décor frames, picture paintings, posters, tapestries, Artwork ', 'profession operation day-to-day trade tools Books ', 'curtains Blinds ', 'control automation Building ', 'dispensers units Built-in-storage ', 'tiles carpet Carpets ', 'business function special performs removal grease Central ', 'storage Cold ', 'transportation mode goods carriage Containers ', 'wiring cables telecommunication Data ', 'appliances) sprinkler security/alarm air-conditioning (e.g. wiring equipment electronic Electrical ', 'themselves) gates doo

In [10]:
# setting up x-features and y-target for model training of text classification model
x_data_final, y_target_final = list(), list()
for item in plant_data:
    x_data_final.append(item)
    y_target_final.append('Plant Classification')
for item in non_plant_data:
    x_data_final.append(item)
    y_target_final.append('Non Plant Classification')
print((x_data_final))
print('')
print('-------')
print('')
print((y_target_final))

['installation conditioning Air ', 'machinery demonstrate need taxpayer businesses, related non-hospitality In satisfied. test premises doubts item served, function legitimate adopted analysis functional décor, mirrors, e.g. identity, separate i.e. quasi-structural, hotels. restaurants clubs, example businesses, hospitality-related function business serve found generally objects stand-alone These murals. decorative including features, ornamental column décor pillar, décor frames, picture paintings, posters, tapestries, Artwork ', 'profession operation day-to-day trade tools Books ', 'curtains Blinds ', 'control automation Building ', 'dispensers units Built-in-storage ', 'tiles carpet Carpets ', 'business function special performs removal grease Central ', 'storage Cold ', 'transportation mode goods carriage Containers ', 'wiring cables telecommunication Data ', 'appliances) sprinkler security/alarm air-conditioning (e.g. wiring equipment electronic Electrical ', 'themselves) gates doo

### 3. Splitting dataset into training and testing

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics import accuracy_score

# standard_scaler = StandardScaler()
# scaled_train_data = standard_scaler.fit_transform(x_data_final)
vectorizer = TfidfVectorizer()
vectorised_train_data = vectorizer.fit_transform(x_data_final)

x_train, x_test, y_train, y_test = train_test_split(vectorised_train_data,
                                                    y_target_final,
                                                    test_size = 0.20,
                                                    random_state = 42)
print(x_train)
print('')
print(x_test)
print('')
print(y_train)
print('')
print(y_test)
print('')

  (0, 20)	0.7071067811865475
  (0, 62)	0.7071067811865475
  (1, 216)	0.38599435352877165
  (1, 221)	0.38599435352877165
  (1, 262)	0.38599435352877165
  (1, 276)	0.38599435352877165
  (1, 128)	0.38599435352877165
  (1, 64)	0.38599435352877165
  (1, 312)	0.3256534266109201
  (2, 125)	1.0
  (3, 49)	0.7401373965680412
  (3, 265)	0.672455674525453
  (4, 37)	0.5773502691896257
  (4, 36)	0.5773502691896257
  (4, 287)	0.5773502691896257
  (5, 74)	0.612824222823564
  (5, 130)	0.306412111411782
  (5, 264)	0.306412111411782
  (5, 14)	0.306412111411782
  (5, 46)	0.306412111411782
  (5, 250)	0.306412111411782
  (5, 168)	0.2783923147480577
  (5, 75)	0.2783923147480577
  (6, 25)	0.6072566762215946
  (6, 15)	0.6072566762215946
  :	:
  (43, 284)	0.12437477000187062
  (43, 76)	0.5475716657928156
  (43, 178)	0.1368929164482039
  (43, 57)	0.1368929164482039
  (43, 280)	0.2737858328964078
  (43, 85)	0.1368929164482039
  (43, 166)	0.1368929164482039
  (43, 97)	0.1368929164482039
  (43, 148)	0.1368929164482

### 4. Model training

#### Using Linear SVM

In [12]:
# create classifier model 
from sklearn.svm import LinearSVC

C_list = [c for c in range(1,101)]
accuracy_list = list()

max_accuracy = 0
ideal_value = 0
for value in C_list:
    TRAINING_MODEL = LinearSVC(C=value,loss='hinge')
    TRAINING_MODEL.fit(x_train,y_train)
    pred_labels = TRAINING_MODEL.predict(x_test)
    correct = 0
    for j in range(len(pred_labels)):
        if pred_labels[j] == y_test[j]:
            correct += 1
    accuracy_percent = correct/len(pred_labels)
    if accuracy_percent > max_accuracy:
        max_accuracy = accuracy_percent
        ideal_value = value
    accuracy_percent = round(accuracy_percent,3) * 100
    accuracy_list.append(accuracy_percent)

result_table = pd.DataFrame(columns=['C value','Accuracy Percent (%)'])
result_table['C value'] = C_list
result_table['Accuracy Percent (%)'] = accuracy_list
result_table.to_csv('results/LINEAR-SVM-Result-1.csv')

#### Performance evaluation for Linear SVM classifier with Cross-Validation

In [14]:
from sklearn.model_selection import cross_val_score, cross_val_predict

number_of_folds = 20
score_1 = cross_val_score(LinearSVC(C = ideal_value,loss='hinge'),
                         x_train,
                         y_train,
                         cv = number_of_folds,
                         scoring = 'accuracy') # cv = 5 means splitting the training dataset into 5 folds, then
# making predictions and evaluating them on each of the 5 folds using the model 
# which is trained on the remaining folds, i.e. 4 folds

score_1 = [round(i,3) *100 for i in (score_1)]

CV_table_1 = pd.DataFrame(columns=['Fold number','Accuracy (%) for Linear SVM classifier'])
CV_table_1['Fold number'] = [i+1 for i in range (number_of_folds)]
CV_table_1['Accuracy (%) for Linear SVM classifier'] = score_1
CV_table_1

C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 16 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)


,Fold number,Accuracy (%) for Linear SVM classifier
0,1,33.3
1,2,0.0
2,3,33.3
3,4,66.7
4,5,100.0
5,6,66.7
6,7,66.7
7,8,66.7
8,9,100.0
9,10,66.7


#### Performance evaluation for Linear SVM classifier with confusion matrix 

In [16]:
from sklearn.metrics import confusion_matrix

y_pred_LinearSVM_classifier = cross_val_predict(LinearSVC(C = ideal_value,loss='hinge'),
                         x_train,
                         y_train,
                         cv = number_of_folds)
score_2 = confusion_matrix(y_train,y_pred_LinearSVM_classifier)
print(score_2)
print('')
TN_LinearSVM_classifier = score_2[0][0]
FP_LinearSVM_classifier = score_2[0][1]
FN_LinearSVM_classifier = score_2[1][0]
TP_LinearSVM_classifier = score_2[1][1]
print('The estimated TN, FP, FN and TP values for the predictions from the SGD classifier are: '
      + str(TN_LinearSVM_classifier) + ', '+ str(FP_LinearSVM_classifier) + ', '+ \
      str(FN_LinearSVM_classifier) + ' and '+ str(TP_LinearSVM_classifier) + 'respectively.')

[[13  3]
 [17 15]]

The estimated TN, FP, FN and TP values for the predictions from the SGD classifier are: 13, 3, 17 and 15respectively.


C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 16 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)


#### Using non-linear SVM: Polynomial Kernel

In [17]:
from sklearn.svm import SVC
import itertools 
C_list = [c for c in range(1,101)]
DEGREE_list = [degree for degree in range(1,10)]
COEF_list = [value for value in range(1,101)]

COMBINATIONS_LIST = [C_list,DEGREE_list,COEF_list]
COMBINATIONS = list(itertools.product(*COMBINATIONS_LIST))
accuracy_list = list()
max_accuracy, ideal_item = 0,0

for item in COMBINATIONS:
    c_value, degree_value, coef_value = item[0], item[1], item[2]
    TRAINING_MODEL = SVC(kernel = 'poly',
                         C = c_value,
                         degree = degree_value,
                         coef0 = coef_value)
    TRAINING_MODEL.fit(x_train,y_train)
    pred_labels = TRAINING_MODEL.predict(x_test)
    correct = 0
    for j in range(len(pred_labels)):
        if pred_labels[j] == y_test[j]:
            correct += 1
    accuracy_percent = correct/len(pred_labels)
    if accuracy_percent >= max_accuracy:
        max_accuracy = accuracy_percent
        ideal_item = item
    accuracy_percent = round(accuracy_percent,3) * 100
    accuracy_list.append(accuracy_percent)
    
result_table_2 = pd.DataFrame(columns=['C value','Degree value','Coef value','Accuracy Percent (%)'])
result_table_2['C value'] = [item[0] for item in COMBINATIONS]
result_table_2['Degree value'] = [item[1] for item in COMBINATIONS]
result_table_2['Coef value'] = [item[2] for item in COMBINATIONS]
result_table_2['Accuracy Percent (%)'] = accuracy_list
result_table_2.to_csv('results/NON-LINEAR-POLYNOMIAL_KERNEL_Result-2.csv')

C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [18]:
result_table_2

,C value,Degree value,Coef value,Accuracy Percent (%)
0,1,1,1,61.5
1,1,1,2,61.5
2,1,1,3,61.5
3,1,1,4,61.5
4,1,1,5,61.5
5,1,1,6,61.5
6,1,1,7,61.5
7,1,1,8,61.5
8,1,1,9,61.5
9,1,1,10,61.5


#### Performance evaluation for Non-linear SVM: Polynomial Kernel with cross validation

In [19]:
from sklearn.model_selection import cross_val_score, cross_val_predict

number_of_folds = 20
ideal_training_model = SVC(kernel = 'poly',
                         C = ideal_item[0],
                         degree = ideal_item[1],
                         coef0 = ideal_item[2])

score_3 = cross_val_score(ideal_training_model,
                         x_train,
                         y_train,
                         cv = number_of_folds,
                         scoring = 'accuracy') # cv = 5 means splitting the training dataset into 5 folds, then
# making predictions and evaluating them on each of the 5 folds using the model 
# which is trained on the remaining folds, i.e. 4 folds

score_3 = [round(i,3) *100 for i in (score_1)]

CV_table_2 = pd.DataFrame(columns=['Fold number','Accuracy (%) for NonLinear Polynomial classifier'])
CV_table_2['Fold number'] = [i+1 for i in range (number_of_folds)]
CV_table_2['Accuracy (%) for NonLinear Polynomial classifier'] = score_1
CV_table_2

C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 16 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)
C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


,Fold number,Accuracy (%) for NonLinear Polynomial classifier
0,1,33.3
1,2,0.0
2,3,33.3
3,4,66.7
4,5,100.0
5,6,66.7
6,7,66.7
7,8,66.7
8,9,100.0
9,10,66.7


#### Performance evaluation for Non-linear SVM: Polynomial Kernel with confusion matrix

In [20]:
from sklearn.metrics import confusion_matrix

y_pred_polynomial_classifier = cross_val_predict(ideal_training_model,
                         x_train,
                         y_train,
                         cv = number_of_folds)
score_4 = confusion_matrix(y_train,y_pred_polynomial_classifier)
print(score_2)
print('')
TN_polynomial_classifier = score_4[0][0]
FP_polynomial_classifier = score_4[0][1]
FN_polynomial_classifier = score_4[1][0]
TP_polynomial_classifier = score_4[1][1]
print('The estimated TN, FP, FN and TP values for the predictions from the SGD classifier are: '
      + str(TN_polynomial_classifier) + ', '+ str(FP_polynomial_classifier) + ', '+ \
      str(FN_polynomial_classifier) + ' and '+ str(TP_polynomial_classifier) + 'respectively.')

[[13  3]
 [17 15]]

The estimated TN, TP, FN and TP values for the predictions from the SGD classifier are: 13, 3, 17 and 15respectively.


C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 16 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)
C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


#### Using non-linear SVM: RBF Gaussian Kernel

In [21]:
from sklearn.svm import SVC
import itertools 
C_list = [c for c in range(1,101)]
GAMMA_list = [g for g in range(1,101)]
COMBINATIONS_LIST = [C_list,GAMMA_list]
COMBINATIONS = list(itertools.product(*COMBINATIONS_LIST))
accuracy_list = list()
max_accuracy, ideal_item = 0,0

for item in COMBINATIONS:
    c_value, gamma_value = item[0], item[1]
    TRAINING_MODEL = SVC(kernel = 'rbf',
                         C = c_value,
                         gamma = gamma_value)
    TRAINING_MODEL.fit(x_train,y_train)
    pred_labels = TRAINING_MODEL.predict(x_test)
    correct = 0
    for j in range(len(pred_labels)):
        if pred_labels[j] == y_test[j]:
            correct += 1
    accuracy_percent = correct/len(pred_labels)
    if accuracy_percent >= max_accuracy:
        max_accuracy = accuracy_percent
        ideal_item = item
    accuracy_percent = round(accuracy_percent,3) * 100
    accuracy_list.append(accuracy_percent)

result_table_3 = pd.DataFrame(columns=['C value','Gamma value','Accuracy Percent (%)'])
result_table_3['C value'] = [item[0] for item in COMBINATIONS]
result_table_3['Gamma value'] = [item[1] for item in COMBINATIONS]
result_table_3['Accuracy Percent (%)'] = accuracy_list
result_table_3.to_csv('results/NON-LINEAR-GAUSSIAN_RBF_KERNEL_Result-3.csv')

In [22]:
result_table_3

,C value,Gamma value,Accuracy Percent (%)
0,1,1,61.5
1,1,2,61.5
2,1,3,61.5
3,1,4,61.5
4,1,5,61.5
5,1,6,61.5
6,1,7,61.5
7,1,8,61.5
8,1,9,61.5
9,1,10,61.5


#### Performance evaluation for Non-linear SVM: RBF Gaussian with cross validation

In [23]:
from sklearn.model_selection import cross_val_score, cross_val_predict

number_of_folds = 20
ideal_training_model = SVC(kernel = 'rbf',
                         C = ideal_item[0],
                         gamma = ideal_item[1])

score_5 = cross_val_score(ideal_training_model,
                         x_train,
                         y_train,
                         cv = number_of_folds,
                         scoring = 'accuracy') # cv = 5 means splitting the training dataset into 5 folds, then
# making predictions and evaluating them on each of the 5 folds using the model 
# which is trained on the remaining folds, i.e. 4 folds

score_5 = [round(i,3) *100 for i in (score_1)]

CV_table_3 = pd.DataFrame(columns=['Fold number','Accuracy (%) for NonLinear Polynomial classifier'])
CV_table_3['Fold number'] = [i+1 for i in range (number_of_folds)]
CV_table_3['Accuracy (%) for NonLinear Polynomial classifier'] = score_1
CV_table_3

C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 16 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)


,Fold number,Accuracy (%) for NonLinear Polynomial classifier
0,1,33.3
1,2,0.0
2,3,33.3
3,4,66.7
4,5,100.0
5,6,66.7
6,7,66.7
7,8,66.7
8,9,100.0
9,10,66.7


#### Performance evaluation for Non-linear SVM: RBF Gaussian with confusion matrix

In [25]:
from sklearn.metrics import confusion_matrix

y_pred_rbf_classifier = cross_val_predict(ideal_training_model,
                         x_train,
                         y_train,
                         cv = number_of_folds)
score_6 = confusion_matrix(y_train,y_pred_polynomial_classifier)
print(score_6)
print('')
TN_rbf_classifier = score_6[0][0]
FP_rbf_classifier = score_6[0][1]
FN_rbf_classifier = score_6[1][0]
TP_rbf_classifier = score_6[1][1]
print('The estimated TN, FP, FN and TP values for the predictions from the SGD classifier are: '
      + str(TN_rbf_classifier) + ', '+ str(FP_rbf_classifier) + ', '+ \
      str(FN_rbf_classifier) + ' and '+ str(TP_rbf_classifier) + 'respectively.')

[[13  3]
 [17 15]]

The estimated TN, FP, FN and TP values for the predictions from the SGD classifier are: 13, 3, 17 and 15respectively.


C:\Users\WZCHE\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 16 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)


####  Using Linear SVM with OvA Classifier

In [26]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

classifier_OVA = OneVsRestClassifier(LinearSVC())
classifier_OVO = OneVsOneClassifier(LinearSVC())

classifier_OVA.fit(x_train,y_train)
pred_labels_OVA = classifier_OVA.predict(x_test)
classifier_OVO.fit(x_train,y_train)
pred_labels_OVO = classifier_OVO.predict(x_test)

correct = 0
for j in range(len(pred_labels_OVA)):
    if pred_labels_OVA[j] == y_test[j]:
        correct += 1
accuracy_percent_OVA = correct/len(pred_labels_OVA)
accuracy_percent_OVA = round(accuracy_percent,3)

correct = 0
accuracy_list = list()
for j in range(len(pred_labels_OVO)):
    if pred_labels_OVO[j] == y_test[j]:
        correct += 1
accuracy_percent_OVO = correct/len(pred_labels_OVO)
accuracy_percent_OVO = round(accuracy_percent,3) 

print('The accuracy percentages for OVA and OVO classifiers are ' + str(accuracy_percent_OVA) + \
     '% and ' + str(accuracy_percent_OVO) + '% respectively.')

The accuracy percentages for OVA and OVO classifiers are 61.5% and 61.5% respectively.
